In [2]:
import copy
import time
import pickle
from collections import Counter 
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable as Variable

import torchvision
import torchvision.transforms as transforms

In [3]:
last_time = None

def start_time():
    global last_time
    last_time = time.time()
def take_time():
    global last_time
    new_time = time.time()
    print("time:", new_time - last_time)
    last_time = new_time

In [4]:
def freq_dict(list):
    
    freq = {}
    for item in list:
        if (item in freq):
            freq[item] += 1
        else:
            freq[item] = 1
    return(freq)

def clip(x, a=0, b=1):
    return(torch.max(torch.min(x, 0*x + b), 0*x + a))

In [6]:
def circuit_output(b, c, print_stuff=False):
#     print(b, c)
    
    n_strings = len(b[0])
    
    gate_i = 0
    cur_width = b.size()[0]
    
    while cur_width > 1:
        if print_stuff:
            print(cur_width)
            print(b)
        
        cur_width = round(cur_width/2)
        outputs = torch.zeros(cur_width, n_strings)

        for out_i in range(cur_width):
            in_i1 = out_i*2
            in_i2 = out_i*2 + 1

            outputs[out_i] = (1 - c[gate_i])*(1 - (1 - b[in_i1])*(1 - b[in_i2])) + c[gate_i]*b[in_i1]*b[in_i2]
            
            gate_i += 1

        b = outputs
    
    return(b[0])

In [7]:
def choose_target(n_bits):
    global c
    c = 1.*(torch.rand(n_bits - 1) > .5)

    while True:
        c = 1.*(torch.randn(n_bits - 1) > 0)
        outputs = [circuit_output(1.*(torch.randn(n_bits, 1) > 0), c) for i in range(1000)]
#         print(sum(outputs))
        if sum(outputs) > 400 and sum(outputs) < 600:
            break
            
    outputs = [circuit_output(1.*(torch.randn(n_bits, 1) > 0), c) for i in range(1000)]
    print("average output of", sum(outputs).item()/1000)
    print(c)
    return(c)

In [8]:
def generate_all_binary_vectors(n_bits, array_so_far=None):
    if array_so_far == None:
        array_so_far = []
#     print(n_bits, array_so_far)
    if n_bits == 0:
        return([array_so_far.copy()])
    
    outputs = []
    
    array_so_far += [0]
    outputs += generate_all_binary_vectors(n_bits - 1, array_so_far)
    
    array_so_far[len(array_so_far) - 1] = 1
    outputs += generate_all_binary_vectors(n_bits - 1, array_so_far)
    
    del array_so_far[-1]
    
    return(outputs)

def generate_random_binary_vectors(n_bits, n):
    return(1.*(torch.rand((n, n_bits)) > .5))

## Initialize and optimize on hypercube

In [49]:
cache = {}

In [72]:
# possible datasets: full, subset, subset_random, random

def run_experiment(c, redundancy, n_epochs, dataset='full', batch_size=100, lr=.003, mom=.999, init_x=None, random_subset_size=10000):
    print('#'*100)
    
    n_bits = len(c) + 1

    all_data = None
    all_data_redundant = None
    
    # if doing true GD, generate the full dataset
    if dataset == 'full':
        all_data = torch.tensor(generate_all_binary_vectors(n_bits))
        all_data_redundant = torch.cat((all_data,)*redundancy, dim=1)

    # if doing subset SGD, generate the subdataset
    if dataset == 'subset' or dataset == 'subset_random':
        all_data = generate_random_binary_vectors(n_bits, random_subset_size)
        all_data_redundant = torch.cat((all_data,)*redundancy, dim=1)
        
    x = init_x
    if x is None:
        x = Variable(torch.rand(redundancy*n_bits - 1), requires_grad=True)
        
    optimizer = optim.SGD([x], lr=lr, momentum=mom)
    # optimizer = optim.Adam([x], lr=.1)

    epochs_since_nonzero_loss = 0
    for epoch in range(n_epochs):
        
        data = None
        data_redundant = None
        if dataset == 'full' or dataset == 'subset':
            data = all_data
            data_redundant = all_data_redundant
        elif dataset == 'subset_random':
            indices = torch.randint(low=0, high=len(all_data), size=(batch_size,))
            data = all_data[indices]
            data_redundant = all_data_redundant[indices]
        elif dataset == 'random':
            data = generate_random_binary_vectors(n_bits, batch_size)
            data_redundant = torch.cat((data,)*redundancy, dim=1)
        else:
            q = 1 + '7'
            
        optimizer.zero_grad()
        output_x = circuit_output(torch.t(data_redundant), x)
        output_c = circuit_output(torch.t(data), c)
        loss = torch.mean(torch.abs(output_x - output_c))

        loss.backward()
        optimizer.step()

        with torch.no_grad():
            x.data = clip(x.data)

        if epoch % 100 == 0:
            print('*'*50, epoch, '*'*50)
            print("loss: ", loss.item())

            x_snapped = 1.*(x > .5)
            output_s = circuit_output(torch.t(data_redundant), x_snapped)
            loss_snapped = torch.mean(torch.abs(output_s - output_c))
            print("loss snapped: ", loss_snapped.item())

            extremeness = torch.sum(torch.min(torch.abs(x), torch.abs(x - 1))).item()
            print("distance to nearest vertex: ", extremeness)
            
#             print(epochs_since_nonzero_loss)
    
        if loss.item() <= 10**-4:
#             print(loss.item())
            epochs_since_nonzero_loss += 1
        else:
            epochs_since_nonzero_loss = 0
        
        if epochs_since_nonzero_loss == 100000:
            print("qqqq", loss.item(), loss.item() <= 10**-4)
            print("ending run - zero loss")
            break
    
    
    if dataset == 'subset' or dataset == 'subset_random':
        x_snapped = 1.*(x > .5)
        
#         indices = torch.randint(low=0, high=len(all_data), size=(100000,))
#         data = all_data[indices]
#         data_redundant = all_data_redundant[indices]
            
#         output_c = circuit_output(torch.t(data), c)
#         output_x = circuit_output(torch.t(data_redundant), x)
#         output_s = circuit_output(torch.t(data_redundant), x_snapped)
        
#         cache['output_c'] = output_c
#         cache['output_x'] = output_x
#         cache['output_s'] = output_s

        output_c = circuit_output(torch.t(all_data), c)
        output_x = circuit_output(torch.t(all_data_redundant), x)
        output_s = circuit_output(torch.t(all_data_redundant), x_snapped)
        loss = torch.mean(torch.abs(output_x - output_c))
        loss_snapped = torch.mean(torch.abs(output_s - output_c))

        print("final loss on train subset:", loss.item(), loss_snapped.item())
    
    data = None
    data_redundant = None
    if dataset == 'full':
        data = all_data
        data_redundant = all_data_redundant
    elif dataset == 'subset' or dataset == 'subset_random':
        data = generate_random_binary_vectors(n_bits, 10*random_subset_size)
        data_redundant = torch.cat((data,)*redundancy, dim=1)
    elif dataset == 'random':
        data = generate_random_binary_vectors(n_bits, 10*random_subset_size)
        data_redundant = torch.cat((data,)*redundancy, dim=1)
    else:
        q = 1 + '7'
    
    output_c = circuit_output(torch.t(data), c)
    x_snapped = 1.*(x > .5)
    output_s = circuit_output(torch.t(data_redundant), x_snapped)
    loss_snapped = torch.mean(torch.abs(output_s - output_c))
    
    print("final model:", x)
    print("final model snapped:", x_snapped)
    
    return loss_snapped.item()

In [95]:
lengths = [4, 8, 16, 32, 64, 128]
cs = {}
for n_bits in lengths:
    cs[n_bits] = choose_target(n_bits)

average output of 0.441
tensor([1., 1., 0.])
average output of 0.523
tensor([0., 0., 0., 0., 0., 1., 1.])
average output of 0.561
tensor([1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0.])
average output of 0.603
tensor([1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
        1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1.])
average output of 0.362
tensor([1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1.,
        1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1.,
        1., 0., 0., 0., 0., 1., 1., 0., 0.])
average output of 0.424
tensor([0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0.,
        1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1.,
        0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1.,

## Experimental rainbow - SGD

In [94]:
torch.manual_seed(0)
run_experiment(cs[64], 4, 1000, dataset='random', batch_size=10, random_subset_size=10000, lr=3, mom=.7)

KeyError: 64

In [97]:
final_losses = {}
n_runs = 20
n_epochs = 1000
batch_size = 10
rdcies = [1, 2, 4, 8]
for n_bits in lengths:
    c = cs[n_bits]
    for rdcy in rdcies:
        print()
        print()
        print('!'*100)
        print(n_bits, "bits, redundancy of", rdcy)
        final_losses[(n_bits, rdcy)] = [run_experiment(c, rdcy, n_epochs, dataset='random', batch_size=batch_size, lr=.5, mom=.7, random_subset_size=10000) for _ in range(n_runs)]



!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
4 bits, redundancy of 1
####################################################################################################
************************************************** 0 **************************************************
loss:  0.2828408479690552
loss snapped:  0.20000000298023224
distance to nearest vertex:  0.8957852125167847
************************************************** 100 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 200 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 300 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 4

************************************************** 200 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 300 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 400 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 500 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 600 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 700 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest

************************************************** 500 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 600 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 700 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 800 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 900 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
final model: tensor([1., 1., 0.], requires_grad=True)
final model snapped: tensor([1., 1., 0.])
###########################################################

distance to nearest vertex:  0.0
final model: tensor([1., 1., 0.], requires_grad=True)
final model snapped: tensor([1., 1., 0.])
####################################################################################################
************************************************** 0 **************************************************
loss:  0.2791180908679962
loss snapped:  0.30000001192092896
distance to nearest vertex:  0.5760866403579712
************************************************** 100 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 200 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 300 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
**************************************************

distance to nearest vertex:  0.0
************************************************** 300 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 400 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 500 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 600 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 700 **************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
************************************************** 800 **************************************************
loss:  0.0
loss 

...

************************************************** 500 **************************************************
loss:  0.29955410957336426
loss snapped:  0.30000001192092896
distance to nearest vertex:  251.45645141601562
************************************************** 600 **************************************************
loss:  0.5001718401908875
loss snapped:  0.5
distance to nearest vertex:  251.50320434570312
************************************************** 700 **************************************************
loss:  0.5004398226737976
loss snapped:  0.5
distance to nearest vertex:  250.64369201660156
************************************************** 800 **************************************************
loss:  0.39920082688331604
loss snapped:  0.4000000059604645
distance to nearest vertex:  250.1168212890625
************************************************** 900 **************************************************
loss:  0.5010941624641418
loss snapped:  0.5
distance to nearest v

************************************************** 0 **************************************************
loss:  0.4564248025417328
loss snapped:  0.4000000059604645
distance to nearest vertex:  257.5573425292969
************************************************** 100 **************************************************
loss:  0.39955657720565796
loss snapped:  0.4000000059604645
distance to nearest vertex:  255.6293487548828
************************************************** 200 **************************************************
loss:  0.4986124038696289
loss snapped:  0.5
distance to nearest vertex:  255.59178161621094
************************************************** 300 **************************************************
loss:  0.3001709282398224
loss snapped:  0.30000001192092896
distance to nearest vertex:  255.22518920898438
************************************************** 400 **************************************************
loss:  0.2998763620853424
loss snapped:  0.300000011920

************************************************** 600 **************************************************
loss:  0.2003742754459381
loss snapped:  0.20000000298023224
distance to nearest vertex:  262.72216796875
************************************************** 700 **************************************************
loss:  0.10807539522647858
loss snapped:  0.10000000149011612
distance to nearest vertex:  262.84124755859375
************************************************** 800 **************************************************
loss:  0.3064432442188263
loss snapped:  0.30000001192092896
distance to nearest vertex:  262.50982666015625
************************************************** 900 **************************************************
loss:  0.49966293573379517
loss snapped:  0.5
distance to nearest vertex:  261.2500915527344
final model: tensor([0.5941, 0.5970, 0.0373,  ..., 1.0000, 1.0000, 0.9541],
       requires_grad=True)
final model snapped: tensor([1., 1., 0.,  ..., 1., 1., 

************************************************** 0 **************************************************
loss:  0.46257853507995605
loss snapped:  0.4000000059604645
distance to nearest vertex:  251.21556091308594
************************************************** 100 **************************************************
loss:  0.8991087079048157
loss snapped:  0.8999999761581421
distance to nearest vertex:  249.4995880126953
************************************************** 200 **************************************************
loss:  0.3045172095298767
loss snapped:  0.30000001192092896
distance to nearest vertex:  249.3636016845703
************************************************** 300 **************************************************
loss:  0.40342530608177185
loss snapped:  0.4000000059604645
distance to nearest vertex:  249.3641357421875
************************************************** 400 **************************************************
loss:  0.5994182825088501
loss snapped: 

************************************************** 600 **************************************************
loss:  0.2288604974746704
loss snapped:  0.20000000298023224
distance to nearest vertex:  229.11500549316406
************************************************** 700 **************************************************
loss:  0.20140382647514343
loss snapped:  0.20000000298023224
distance to nearest vertex:  226.17213439941406
************************************************** 800 **************************************************
loss:  0.3003183901309967
loss snapped:  0.30000001192092896
distance to nearest vertex:  225.1256866455078
************************************************** 900 **************************************************
loss:  0.08367845416069031
loss snapped:  0.10000000149011612
distance to nearest vertex:  224.5872802734375
final model: tensor([0.6138, 0.4306, 0.8482,  ..., 1.0000, 1.0000, 0.0000],
       requires_grad=True)
final model snapped: tensor([1., 0., 

In [98]:
for params in sorted(final_losses):
    print(params[0], "bits and redundancy of", params[1], "gave average loss of", "%.5f"%torch.Tensor(final_losses[params]).mean().item(), "giving nonzero loss with probability", "%.2f"%(1 - sum(torch.Tensor(final_losses[params]) == 0).item()*1.0/n_runs))
    
for params in sorted(final_losses):
    print(params[0], params[1], "%.5f"%torch.Tensor(final_losses[params]).mean().item(), "%.2f"%(1 - sum(torch.Tensor(final_losses[params]) == 0).item()*1.0/n_runs))

4 bits and redundancy of 1 gave average loss of 0.00000 giving nonzero loss with probability 0.00
4 bits and redundancy of 2 gave average loss of 0.01244 giving nonzero loss with probability 0.05
4 bits and redundancy of 4 gave average loss of 0.00000 giving nonzero loss with probability 0.00
4 bits and redundancy of 8 gave average loss of 0.00000 giving nonzero loss with probability 0.00
8 bits and redundancy of 1 gave average loss of 0.02206 giving nonzero loss with probability 0.35
8 bits and redundancy of 2 gave average loss of 0.01098 giving nonzero loss with probability 0.25
8 bits and redundancy of 4 gave average loss of 0.00467 giving nonzero loss with probability 0.15
8 bits and redundancy of 8 gave average loss of 0.00000 giving nonzero loss with probability 0.00
16 bits and redundancy of 1 gave average loss of 0.06164 giving nonzero loss with probability 0.60
16 bits and redundancy of 2 gave average loss of 0.01782 giving nonzero loss with probability 0.35
16 bits and redund

In [103]:
final_losses[(128, 4)]

[0.1923699975013733,
 0.16798999905586243,
 0.09775999933481216,
 0.4209800064563751,
 0.26798999309539795,
 0.4186199903488159,
 0.0561399981379509,
 0.4165399968624115,
 0.41894999146461487,
 0.08844999969005585,
 0.4145199954509735,
 0.1795099973678589,
 0.31198999285697937,
 0.41887998580932617,
 0.4194500148296356,
 0.23699000477790833,
 0.07355999946594238,
 0.42089998722076416,
 0.4183900058269501,
 0.20395000278949738]

In [106]:
print(x)
y = torch.Tensor([1, 0, 0, 0, 0, 0, 0, 0])

output_x = circuit_output(torch.t(data_doubled), x)
output_y = circuit_output(torch.t(data_doubled), y)
output_c = circuit_output(torch.t(data), c)

print(sum(abs(output_x - output_c)))
print(sum(abs(output_y - output_c)))

# print(data_doubled)
# print(output_y)
# print(data_doubled[4:5])
# print('*'*100)
# print(circuit_output(torch.t(data_doubled[4:5]), y, print_stuff=True))

tensor([ 1.0000e+00, -5.4456e-06,  4.9505e-04,  7.5124e-01,  6.1709e-01,
         9.3323e-01,  8.1097e-01], requires_grad=True)
tensor(5.7244, grad_fn=<AddBackward0>)
tensor(6.)


In [135]:
x = torch.Tensor([0, 1, 0])

output_x = circuit_output(torch.t(data), x)
output_c = circuit_output(torch.t(data), c)

print(sum(abs(output_x - output_c)))

tensor(4.)


In [778]:
print(x)
print(c)

tensor([6.8733e-04, 1.0004e+00, 1.3043e-02], requires_grad=True)
tensor([0., 0., 1.])


In [385]:
x_orig = x
x_clipped = 1.*(x > .5)
x

tensor([0.9105, 0.6759, 1.0000, 0.9095, 0.0000, 0.0000, 1.0000, 1.0000, 0.0236,
        0.0000, 0.0000, 0.0000, 0.9949, 0.9949, 0.9748, 0.9748, 0.3941, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.9849, 0.0000, 1.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.8900, 1.0000], requires_grad=True)

In [699]:
data = all_data

x_tweaked = x_clipped + torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
print(x_tweaked)

output_x = circuit_output(torch.t(data), x_clipped)
output_x_t = circuit_output(torch.t(data), x_tweaked)
output_c = circuit_output(torch.t(data), c)

loss = torch.mean(torch.abs((output_x - output_c)))
loss_t = torch.mean(torch.abs((output_x_t - output_c)))

print(loss_t - loss)

tensor([0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1.])
tensor(0.)


In [667]:
c - x_clipped

tensor([ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.])

In [ ]:
# -> there are local mins besides the global min

In [852]:
# check if adding more gates besides AND/OR can help escape a local minimum

all_data = generate_all_binary_vectors(4)

def target_f(x):
    o1 = x[0] or x[1]
    o2 = x[2] or x[3]
    o3 = o1 and o2
    return(o3)

def current_f(x):
    o1 = x[0] or x[1]
    o2 = .8*(x[2] and x[3]) + .2*(x[2] or x[3])
    o3 = .99*o1 + .01*(o1 and o2)
    return(o3)

sum([abs(target_f(x) - current_f(x)) for x in all_data])

3.018

## Check for local minima

In [339]:
def bitstring_neighbors(b):
    neighbors = torch.cat((b,)*len(b[0]))
    neighbors = neighbors + torch.eye(len(b[0]))
    neighbors %= 2
    return(neighbors)

def binary_to_decimal(b):
    nb = len(b[0])
    x = torch.Tensor([[2**k for k in range(nb - 1, -1, -1)]])
    output = torch.mm(1.*b, 1.*torch.t(x))
    return(output.long())

def is_local_min(b, errors, strict=True):
    error = errors[binary_to_decimal(b).view(-1)[0]]
    neighbors = bitstring_neighbors(b)
    neighbor_is = binary_to_decimal(neighbors).view(-1).tolist()
    neighbor_errors = errors[neighbor_is]
    min_neighbor_error = min(neighbor_errors)
    
    if error < min_neighbor_error:
        return(True)
    elif error == min_neighbor_error:
        if strict:
            return(False)
        return(True)
    return(False)

def get_equivalent_neighbors_if_min(b, errors):
    error = errors[binary_to_decimal(b).view(-1)[0]]
    neighbors = bitstring_neighbors(b)
    neighbor_is = binary_to_decimal(neighbors).view(-1).tolist() + binary_to_decimal(b)[0].tolist()
    neighbor_errors = errors[neighbor_is]
    min_neighbor_error = min(neighbor_errors)
    
    if min_neighbor_error < error:
        return(set())
    
    output = set()
    for i in range(len(neighbor_is)):
        if neighbor_errors[i] == error:
            output.add(neighbor_is[i])
    return(output)

In [579]:
n_bits = 16
redundancy = 1
n_gates = redundancy*n_bits - 1

c = cs[n_bits]
# c = torch.Tensor([0, 0, 1])
# c = torch.Tensor([0, 0, 0, 0, 1, 1, 0])
# c = torch.Tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1])

data = torch.tensor(generate_all_binary_vectors(n_bits))
if redundancy > 1:
    data = torch.cat((data,)*redundancy, dim=1)

circuits = torch.tensor(generate_all_binary_vectors(n_gates))

errors = torch.zeros(len(circuits))
for i in range(len(circuits)):
    output = circuit_output(torch.t(data), circuits[i])
    target = circuit_output(torch.t(data), c)
    errors[i] = sum(abs(output - target))

print(errors)

KeyboardInterrupt: 

### Find single local mins

In [ ]:
for i in range(len(circuits)):
    if is_local_min(circuits[i:(i + 1)], errors, strict=True):
        print(i, circuits[i], errors[i])

### Find local min clusters

In [ ]:
cohort_is = [get_equivalent_neighbors_if_min(circuits[i:(i + 1)], errors) for i in range(len(circuits))]

original_cohort_is = copy.deepcopy(cohort_is)

for run in range(20):
    for i in range(len(circuits)):
        set_i = cohort_is[i]
        for j in set_i:
            if cohort_is[j] == set():
                cohort_is[i] = set()
                break

cluster_ids = [str(i) for i in range(len(circuits))]
for run in range(20):
    for i in range(len(circuits)):
        if cohort_is[i] == set():
            cluster_ids[i] = ''
        else:
            for j in cohort_is[i]:
                cluster_ids[j] = cluster_ids[i]

zero_cluster_ids = set()
min_cluster_ids = set()
for ci in set(cluster_ids):
    if ci != '':
        if errors[int(ci)] == 0:
            zero_cluster_ids.add(ci)
        else:
            min_cluster_ids.add(ci)

cluster_size_dict = freq_dict(cluster_ids)
min_cluster_errors = [errors[int(ci)].item() for ci in min_cluster_ids]
min_cluster_sizes = [cluster_size_dict[mci] for mci in min_cluster_ids]
zero_cluster_sizes = [cluster_size_dict[zci] for zci in zero_cluster_ids]
            
print(str(len(set(zero_cluster_ids))) + " zero clusters with sizes " + str(zero_cluster_sizes))
print(str(len(set(min_cluster_ids))) + " min clusters with sizes " + str(min_cluster_sizes) + " and errors " + str(min_cluster_errors))

print(cluster_ids)
print(cohort_is)


In [358]:
original_cohort_is

[{0, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384},
 {0, 1, 3, 5, 9, 17, 33, 65, 129, 257, 513, 1025, 2049, 4097, 8193, 16385},
 {0, 2, 3, 6, 10, 18, 34, 66, 130, 258, 514, 1026, 2050, 4098, 8194, 16386},
 set(),
 {0, 4, 5, 6, 12, 20, 36, 68, 132, 260, 516, 1028, 2052, 4100, 8196, 16388},
 set(),
 {2, 4, 6, 7, 14, 22, 38, 70, 134, 262, 518, 1030, 2054, 4102, 8198, 16390},
 set(),
 {0, 8, 9, 10, 12, 24, 40, 72, 136, 264, 520, 1032, 2056, 4104, 8200, 16392},
 set(),
 set(),
 {11, 15, 27, 43, 75, 523, 1035, 2059, 4107, 8203, 16395},
 {4, 8, 12, 13, 14, 28, 44, 76, 140, 268, 524, 1036, 2060, 4108, 8204, 16396},
 set(),
 set(),
 {11, 15, 31, 47, 79, 527, 1039, 2063, 4111, 8207, 16399},
 {0, 16, 17, 18, 20, 24, 48, 80, 144, 272, 528, 1040, 2064, 4112, 8208, 16400},
 set(),
 set(),
 {19, 23, 27, 51, 83, 147, 275, 2067, 4115, 8211, 16403},
 {4, 16, 20, 21, 22, 28, 52, 84, 148, 276, 532, 1044, 2068, 4116, 8212, 16404},
 set(),
 set(),
 {19, 23, 31, 55, 87, 151, 279, 2071